In [ ]:
import pandas as pd
from constants import NOMINAL_COLUMNS, DISCRETE_COLUMNS, CONTINUOUS_COLUMNS
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from MLModel import MLModel
import utils

In [ ]:
model_handler = MLModel()

fill_values_nominal = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_nominal.pkl')
fill_values_discrete = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_discrete.pkl')
fill_values_continuous = model_handler.load_model('artifacts/nan_outlier_handler/fill_values_continuous.pkl')
min_max_scaler_dict = model_handler.load_model('artifacts/encoders/min_max_scaler_dict.pkl')
onehot_encoders = model_handler.load_model('artifacts/encoders/onehot_encoders_dict.pkl')
model = model_handler.load_model('artifacts/models/xgb_model.pkl')

In [ ]:
data_path = 'data/horse-colic-train.csv'
df = pd.read_csv(data_path)

In [ ]:
sample_data = df.iloc[0]
sample_data = pd.DataFrame([sample_data])

In [ ]:
# sample_data.drop(columns=['Unnamed: 0'], inplace=True)
# sample_data.drop(columns=['flight'], inplace=True)

for col in CONTINUOUS_COLUMNS:  
    sample_data[col] = pd.to_numeric(sample_data[col], errors='coerce')

for col in NOMINAL_COLUMNS:
    sample_data[col].fillna(fill_values_nominal[col], inplace=True)

for col in DISCRETE_COLUMNS:
    sample_data[col].fillna(fill_values_discrete[col], inplace=True)

for col in CONTINUOUS_COLUMNS:
    sample_data[col].fillna(fill_values_continuous[col], inplace=True)

for col, encoder in onehot_encoders.items():
    new_data = encoder.transform(sample_data[col].to_numpy().reshape(-1, 1))
    new_df = pd.DataFrame(new_data, columns=encoder.get_feature_names_out([col]))
    sample_data = pd.concat([sample_data, new_df], axis=1).drop(columns=[col])

for col, scaler in min_max_scaler_dict.items():
    if col in sample_data.columns:
        sample_data[col] = scaler.transform(sample_data[[col]])

if 'price' in sample_data.columns:
    sample_data = sample_data.drop(columns=['price'])

predicted_value = model.predict(sample_data)

print(f"The predicted value is: {predicted_value[0]}")